In [1]:
import findspark
findspark.init('/home/yerke/spark-3.3.0-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [6]:
training = spark.read.format('libsvm').load('../course_data/sample_linear_regression_data.txt')

In [7]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [8]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [9]:
lrModel = lr.fit(training)

In [11]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [14]:
training_summary = lrModel.summary

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [16]:
all_data = spark.read.format('libsvm').load('../course_data/sample_linear_regression_data.txt')

In [17]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                351|
|   mean|  0.289209274405542|
| stddev| 10.773478277343388|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                150|
|   mean| 0.1812589640222966|
| stddev|   9.19746055795193|
|    min|-21.432387764165806|
|    max| 22.647750304177556|
+-------+-------------------+



In [25]:
correct_model = lr.fit(train_data)

In [26]:
test_results = correct_model.evaluate(test_data)

In [27]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -22.06996683910152|
| -20.08178847395746|
|-20.660407083667824|
| -16.75822607243078|
|-15.915604784195853|
|-19.283514620802553|
| -16.93512068309126|
|-13.907110427446005|
|-16.824425780185756|
|-13.889036982667227|
| -17.71102906552695|
|-18.587740013400307|
| -14.39537923278512|
|-14.742983452100342|
| -16.82556166247272|
| -9.158802739362265|
|-10.888737417813378|
|  -6.29081886588222|
|-10.688813590884072|
|-11.352286454643979|
+-------------------+
only showing top 20 rows



In [28]:
test_results.rootMeanSquaredError

9.252360854156324

In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = correct_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 0.6375790749357153|
|(10,[0,1,2,3,4,5,...|0.19722769968403442|
|(10,[0,1,2,3,4,5,...| 0.7874160455994177|
|(10,[0,1,2,3,4,5,...|-2.4100665505329832|
|(10,[0,1,2,3,4,5,...| -1.411115948480095|
|(10,[0,1,2,3,4,5,...| 2.2181149949265366|
|(10,[0,1,2,3,4,5,...| 0.2160238494861705|
|(10,[0,1,2,3,4,5,...| -2.178548613575485|
|(10,[0,1,2,3,4,5,...| 1.1009101671371848|
|(10,[0,1,2,3,4,5,...|-1.5483478107639905|
|(10,[0,1,2,3,4,5,...|  2.351484185694272|
|(10,[0,1,2,3,4,5,...| 3.2529725334779642|
|(10,[0,1,2,3,4,5,...|-0.3673790201460064|
|(10,[0,1,2,3,4,5,...| 1.7030553879957264|
|(10,[0,1,2,3,4,5,...|  3.903338559102298|
|(10,[0,1,2,3,4,5,...|-3.6144242598889313|
|(10,[0,1,2,3,4,5,...| -1.016249484861737|
|(10,[0,1,2,3,4,5,...| -5.349730812006606|
|(10,[0,1,2,3,4,5,...|-0.2571060668988607|
|(10,[0,1,2,3,4,5,...| 0.7521561127349456|
+----------